In [1]:
# Aqui se debe poner la carpeta de la computadora local
working_directory = "../..//normanbuck/datasets/"
if (getwd() != working_directory) {
  setwd(working_directory)
}


# load library to read csv.gz
library(data.table)


## Carga micro dataset
PARAM <- list()
PARAM$input$dataset <- "competencia_02_3.csv.gz"
# cargo el dataset donde voy a entrenar el modelo
dataset3 <- fread(PARAM$input$dataset)

## expolratory data analysis
head(dataset3)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,⋯,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71,CONTINUA
29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,⋯,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87,CONTINUA
29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,⋯,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00,CONTINUA
29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,⋯,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31,CONTINUA
29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,⋯,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66,CONTINUA
29188277,201901,1,0,1,65,296,1672.70,16113.26,796.59,⋯,1,0.00,-5791.17,0.00,7827,12539.39,6,0,797.64,CONTINUA


In [23]:
max(dataset3[,foto_mes])

[1] 201901

In [2]:
## get columns that are numeric
numeric_columns <- sapply(dataset3, is.double)
class(numeric_columns)

[1] "logical"

In [3]:
## select numeric columns and columnn numero_de_cliente and foto_mes
dataset3_numeric <- dataset3[, numeric_columns, with= FALSE]
## cbind numero_de_cliente and foto_mes
dataset3_numeric <- cbind(dataset3[, c("numero_de_cliente", "foto_mes")], dataset3_numeric)

dataset3_numeric[1:5, , ]

numero_de_cliente,foto_mes,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,mcuenta_corriente_adicional,mcuenta_corriente,mcaja_ahorro,⋯,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_mconsumototal,Visa_mpagominimo
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29183981,201901,8304.31,42993.54,2501.04,-2325.76,7273.13,0,0.00,135427.60,⋯,36602.13,24.92,603215.2,0,0,0.00,-35773.32,2.33,36602.13,5008.71
29184630,201901,18318.76,42597.26,14986.76,321.62,2518.96,0,27669.01,16206.53,⋯,28503.06,75.49,458226.8,0,0,0.00,-67414.82,1149.57,28503.06,1395.87
29185587,201901,20862.84,85481.31,4228.40,21.29,14159.82,0,-10425.92,86.40,⋯,5170.25,0.00,597328.2,0,0,0.00,0.00,0.00,5170.25,0.00
29185646,201901,25218.94,165317.78,4474.25,-2978.40,20663.56,0,0.00,167672.20,⋯,67991.90,7.74,598054.1,0,0,69653.59,-59422.00,0.00,67991.90,2897.31
29187499,201901,5307.54,32374.08,4706.22,-160.11,672.74,0,-3118.35,2787.00,⋯,4851.93,0.00,28152.0,0,0,0.00,-4802.21,0.00,4851.93,492.66


### shift own

In [4]:
dt <- data.table(
  numero_de_cliente = c(1, 1, 1, 2, 2, 2, 3, 3, 3),
  foto_mes = c(202101, 202102, 202103, 202201, 202202, 202103, 202301, 202302, 202103),
  value = c(10, 20, 30, 40, 50, 60, -10, -5, 9)
)
dt

numero_de_cliente,foto_mes,value
<dbl>,<dbl>,<dbl>
1,202101,10
1,202102,20
1,202103,30
2,202201,40
2,202202,50
2,202103,60
3,202301,-10
3,202302,-5
3,202103,9


In [5]:
# Shift the "value" column to take the previous "foto_mes" value within each group
meses_atras <- 2
dt[, shifted_value := shift(value, type = "lag", fill = NA, n = meses_atras), by = numero_de_cliente]

# Print the modified data.table
dt

numero_de_cliente,foto_mes,value,shifted_value
<dbl>,<dbl>,<dbl>,<dbl>
1,202101,10,NA
1,202102,20,NA
1,202103,30,10
2,202201,40,NA
2,202202,50,NA
2,202103,60,40
3,202301,-10,NA
3,202302,-5,NA
3,202103,9,-10


Pruebo lo mismo sobre varias columnas.

In [6]:
dt <- data.table(
  numero_de_cliente = c(1, 1, 1, 2, 2, 2, 3, 3, 3),
  foto_mes = c(202101, 202102, 202103, 202201, 202202, 202103, 202301, 202302, 202103),
  value = c(10, 20, 30, 40, 50, 60, -10, -5, 9),
  value2 = c(100, 200, 300, 400, 500, 600, -100, -50, 90)
)
dt

numero_de_cliente,foto_mes,value,value2
<dbl>,<dbl>,<dbl>,<dbl>
1,202101,10,100
1,202102,20,200
1,202103,30,300
2,202201,40,400
2,202202,50,500
2,202103,60,600
3,202301,-10,-100
3,202302,-5,-50
3,202103,9,90


In [7]:
# same shift as above but for all columns except the numero_de_cliente, foto_mes
columns_to_shift <- names(dt)[!names(dt) %in% c("numero_de_cliente", "foto_mes")]
columns_to_shift

[1] "value"  "value2"

In [8]:
a <- c('value', 'value2')
class(a)

[1] "character"

In [9]:
meses_atras <- 2
shifted_names <- paste0(columns_to_shift, "_lag", meses_atras)
shifted_names

[1] "value_lag2"  "value2_lag2"

In [11]:
meses_atras <- 2
dt[,
  (shifted_names) := lapply(
    .SD, function(x) shift(x, type = "lag", fill = NA, n = meses_atras)
  ),
  by = numero_de_cliente, .SDcols = columns_to_shift
]
dt

numero_de_cliente,foto_mes,value,value2,value_lag2,value2_lag2
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,202101,10,100,NA,NA
1,202102,20,200,NA,NA
1,202103,30,300,10,100
2,202201,40,400,NA,NA
2,202202,50,500,NA,NA
2,202103,60,600,40,400
3,202301,-10,-100,NA,NA
3,202302,-5,-50,NA,NA
3,202103,9,90,-10,-100


In [20]:
nrow(dataset)

[1] 124864

In [22]:
min(dataset[, foto_mes])

[1] 201901

In [19]:
# 2.o intento sobre dataset original
dataset <- dataset3
setorder(dataset, numero_de_cliente, foto_mes)
dataset[95000:95005, ]

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
106347392,201901,1,0,1,29,99,1077.59,15521.11,582.80,⋯,1,0.00,-5663.20,1,2096,16079.54,14,0,1008.78,CONTINUA
106348553,201901,1,0,1,31,99,698.01,12562.12,744.57,⋯,8,32657.56,-18812.36,0,2885,20599.15,15,0,10521.81,CONTINUA
106350859,201901,1,0,1,35,73,3341.23,21554.95,1274.10,⋯,1,0.00,-6403.93,0,2206,10235.21,7,0,633.42,CONTINUA
106351252,201901,1,0,1,36,53,2851.91,14286.55,691.82,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,CONTINUA
106352874,201901,1,0,1,37,99,1327.23,15162.44,408.14,⋯,8,0.00,NA,NA,2942,NA,NA,NA,0.00,CONTINUA
106356273,201901,1,0,1,57,39,2388.63,24015.88,1649.66,⋯,1,0.00,-16140.48,0,1187,6887.52,11,0,16363.35,CONTINUA


In [12]:
# todo junto sobre el dataset3_numeric
dataset <- dataset3_numeric
setorder(dataset, numero_de_cliente, foto_mes)
columns_to_shift <- names(dataset)[!names(dataset) %in% c("numero_de_cliente", "foto_mes", "clase_ternaria")]
meses_atras <- 1
columns_by_shift <- paste0(columns_to_shift, "_lag", meses_atras)
dataset[,
  (columns_by_shift) := lapply(
    .SD, function(x) shift(x, type = "lag", fill = NA, n = meses_atras)
  ),
  by = numero_de_cliente, .SDcols = columns_to_shift
]
dataset[1:5,]

In [14]:
dataset[,mrentabilidad_lag_1]

[1] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
   [25] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
   [49] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
   [73] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
   [97] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [121] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [145] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [169] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [193] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [217] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [241] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [265] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [289] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [313] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [337] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [361] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [385] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [409] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [433] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [457] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [481] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [505] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [529] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [553] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [577] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [601] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [625] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [649] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [673] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [697] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [721] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [745] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [769] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [793] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [817] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [841] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [865] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [889] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [913] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [937] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [961] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  [985] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [1009] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [1033] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [1057] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [1081] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [1105] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [1129] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [1153] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [1177] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [12

## id

In [4]:
# names of columns except numero_de_cliente, foto_mes and clase_ternaria
all_columns <- setdiff(
  colnames(dataset3),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
)
dataset3[1:5,]

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,⋯,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71,CONTINUA
29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,⋯,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87,CONTINUA
29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,⋯,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00,CONTINUA
29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,⋯,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31,CONTINUA
29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,⋯,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66,CONTINUA


In [5]:
# orders first by numero_de_cliente and then by foto_mes
setorder(dataset3, numero_de_cliente, foto_mes)
dataset3[1:5,]

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,⋯,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71,CONTINUA
29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,⋯,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87,CONTINUA
29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,⋯,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00,CONTINUA
29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,⋯,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31,CONTINUA
29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,⋯,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66,CONTINUA


In [14]:
periods <- seq(1:2)
for (i in periods){
  lagcolumns <- paste("lag", all_columns, i, sep = ".")
  dataset3[, (lagcolumns) := shift(.SD, type = "lag", fill = NA, n = i), .SDcols = all_columns,  by = numero_de_cliente]
}

In [15]:
dataset3[1:5,]

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,lag_Visa_madelantodolares_2,lag_Visa_fultimo_cierre_2,lag_Visa_mpagado_2,lag_Visa_mpagospesos_2,lag_Visa_mpagosdolares_2,lag_Visa_fechaalta_2,lag_Visa_mconsumototal_2,lag_Visa_cconsumos_2,lag_Visa_cadelantosefectivo_2,lag_Visa_mpagominimo_2
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>
29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


## Unsere

In [23]:
# for each numero_de_cliente difference between foto_mes for each numeric column
dataset3_numeric_diff <- dataset3_numeric[, lapply(.SD, diff), by = numero_de_cliente, .SDcols = -c("numero_de_cliente", "foto_mes")]



In [19]:
dataset3_numeric_diff[1:5, , ]

numero_de_cliente,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,mcuenta_corriente_adicional,mcuenta_corriente,mcaja_ahorro,mcaja_ahorro_adicional,⋯,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_mconsumototal,Visa_mpagominimo
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
